#### xml 파일 알람 이벤트 시작 진행 시간 가져오기
- StartTime, AlarmDuration

In [1]:
import os 
from pathlib import Path
import glob
import cv2
import xml.etree.ElementTree as ET

xml_file = "./data_sample/C055200_006.xml"
doc = ET.parse(xml_file)

# root 노드 가져오기
root = doc.getroot()

for Alarm in root.findall("./Library/Clip/Alarms/Alarm"):
    start = Alarm.find('StartTime').text
    AlarmDuration = Alarm.find('AlarmDuration').text
    AlarmDescription = Alarm.find('AlarmDescription').text

print(start)
print(AlarmDuration)

00:04:00
0:0:10


In [2]:
StartHour = int(start[3:5])
StartMin = int(start[6:8])
AlarmDuration_int = int(AlarmDuration[4:6])

print(StartHour)
print(StartMin)
print(AlarmDuration_int)

4
0
10


In [3]:
StartTime_time = (StartHour * 60) + StartMin
print(StartTime_time)

240


#### opencv 사용하여 해당 영상 fps 확인

In [4]:
import cv2

# mp4 영상 위치
mp4_file = './data_sample/C055200_006.mp4'
cap = cv2.VideoCapture(mp4_file)

width = float(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = float(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
count = float(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print('Frame width:', width)
print('Frame height:', height)
print('Frame count:', count)

fps = float(cap.get(cv2.CAP_PROP_FPS))

print('FPS:', fps)

Frame width: 1280.0
Frame height: 720.0
Frame count: 16320.0
FPS: 30.000027573554757


In [5]:
Start_fps = StartTime_time * fps
End_fps = (StartTime_time + AlarmDuration_int) * fps

print("영상 시작 프레임 : ", Start_fps)
print("영상 끝나는 프레임 : ", End_fps)

영상 시작 프레임 :  7200.006617653142
영상 끝나는 프레임 :  7500.00689338869


#### 디렉토리

In [7]:
import os 
import os.path
import glob

# data_sample 주소 입력
path = 'C:\OpenCV\data_sample'

file_list1 = os.listdir(path)

print(file_list1[0])

C045100_001.mp4


In [8]:
from pathlib import Path

xml_path = glob.glob(f"{path}/*.xml", recursive=True)
mp4_path = glob.glob(f"{path}/*.mp4", recursive=True)

xmlfile_name = Path(xml_path[0]).stem
mp4file_name = Path(xml_path[0]).stem

print(xml_path)

print(xmlfile_name)
print(mp4file_name)

['C:\\OpenCV\\data_sample\\C045100_001.xml', 'C:\\OpenCV\\data_sample\\C045300_010.xml', 'C:\\OpenCV\\data_sample\\C055200_006.xml']
C045100_001
C045100_001


In [9]:
for xml_file in xml_path :
    

    mp4_file = xml_file[:-4]+".mp4"

    mp4_file_name = Path(mp4_file).stem
    
    cap = cv2.VideoCapture(mp4_file)

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))
    

    # 동영상 FPS값
    fps = float(cap.get(cv2.CAP_PROP_FPS))

    print(f'{mp4_file_name}의 ', 'Frame width:', width)
    print(f'{mp4_file_name}의 ', 'Frame height:', height)
    print(f'{mp4_file_name}의 ', 'Frame count:', count)

    print(f'{mp4_file_name}의 ', 'FPS:', fps)
    print(f'{mp4_file_name}의 ', 'codec:', fourcc)

C045100_001의  Frame width: 1280
C045100_001의  Frame height: 720
C045100_001의  Frame count: 14100
C045100_001의  FPS: 30.000025531936622
C045100_001의  codec: 828601953
C045300_010의  Frame width: 1280
C045300_010의  Frame height: 720
C045300_010의  Frame count: 14340
C045300_010의  FPS: 30.000027196677376
C045300_010의  codec: 828601953
C055200_006의  Frame width: 1280
C055200_006의  Frame height: 720
C055200_006의  Frame count: 16320
C055200_006의  FPS: 30.000027573554757
C055200_006의  codec: 828601953


In [10]:
def time2sec(timeStr):
    
    time_HMS = timeStr.split(":")
    hour = int(time_HMS[0])
    min = int(time_HMS[1])
    sec = int(time_HMS[2])
    
    return hour*3600+min*60+sec

In [11]:
i = 0

while i < (len(xml_path)):
    xml_path[i]
    mp4_path[i]
    i += 1

print(xml_path)
print(mp4_path)

['C:\\OpenCV\\data_sample\\C045100_001.xml', 'C:\\OpenCV\\data_sample\\C045300_010.xml', 'C:\\OpenCV\\data_sample\\C055200_006.xml']
['C:\\OpenCV\\data_sample\\C045100_001.mp4', 'C:\\OpenCV\\data_sample\\C045300_010.mp4', 'C:\\OpenCV\\data_sample\\C055200_006.mp4']


### result 폴더 만들기

In [12]:
result_folder = f'{path}/result'

if not os.path.isdir(result_folder):
    os.mkdir(result_folder)

#### FFMPEG Practice

In [ ]:
from tkinter import S
import ffmpeg
import sys

# sys.path.append(r'C:\Program Files\ffmpeg-4.3\bin')h

stream = ffmpeg.input(mp4_file) # video location

# 시작 종료 시간으로 영상 편집
stream = stream.trim(start = StartTime_time, duration = AlarmDuration).filter('setpts', 'PTS-STARTPTS')
stream = stream.filter('fps', fps, round='up').filter('scale', width, height)

stream = ffmpeg.output(stream, f"{result_folder}/test.mp4")

ffmpeg.run(stream)

In [51]:
from tkinter import S
import ffmpeg
import sys

# sys.path.append(r'C:\Program Files\ffmpeg-4.3\bin')h

stream = ffmpeg.input(mp4_file) # video location

# 시작 종료 프레임 으로 영상 편집
stream = stream.trim(start_frame=6150.005575318862, end_frame=6450.005847285636).filter('setpts', 'PTS-STARTPTS')
stream = stream.filter('fps', fps, round='up').filter('scale', width, height)

stream = ffmpeg.output(stream, f"{result_folder}/test.mp4")

ffmpeg.run(stream)

(None, None)

#### 비디오를 프레임 단위로 자르는 코드(이미지 저장)

In [23]:
from pathlib import Path
import cv2
from cv2 import imwrite


edit_result_path = glob.glob(f"{result_folder}/*.mp4", recursive=True)


i = 0

while i < (len(edit_result_path)):

    edit_result_name = Path(edit_result_path[i]).stem
    editimage_result_folder = f'{result_folder}/IMAGE_of_{edit_result_name}'


    # 이미지 결과 폴더 생성
    if not os.path.isdir(editimage_result_folder):
        os.mkdir(editimage_result_folder)


    # 영상 이미지 자르기
    vidcap = cv2.VideoCapture(f"{edit_result_path[i]}")
    success,image = vidcap.read()

    count = 1
    success = True

    while success:
        cv2.imwrite(f'{editimage_result_folder}/' + '%d.jpg' % count, image)     # save frame as JPG file
        success,image = vidcap.read()
        print("saved image %d.jpg" % count)
        
        count += 1
        
    i += 1

saved image 1.jpg
saved image 2.jpg
saved image 3.jpg
saved image 4.jpg
saved image 5.jpg
saved image 6.jpg
saved image 7.jpg
saved image 8.jpg
saved image 9.jpg
saved image 10.jpg
saved image 11.jpg
saved image 12.jpg
saved image 13.jpg
saved image 14.jpg
saved image 15.jpg
saved image 16.jpg
saved image 17.jpg
saved image 18.jpg
saved image 19.jpg
saved image 20.jpg
saved image 21.jpg
saved image 22.jpg
saved image 23.jpg
saved image 24.jpg
saved image 25.jpg
saved image 26.jpg
saved image 27.jpg
saved image 28.jpg
saved image 29.jpg
saved image 30.jpg
saved image 31.jpg
saved image 32.jpg
saved image 33.jpg
saved image 34.jpg
saved image 35.jpg
saved image 36.jpg
saved image 37.jpg
saved image 38.jpg
saved image 39.jpg
saved image 40.jpg
saved image 41.jpg
saved image 42.jpg
saved image 43.jpg
saved image 44.jpg
saved image 45.jpg
saved image 46.jpg
saved image 47.jpg
saved image 48.jpg
saved image 49.jpg
saved image 50.jpg
saved image 51.jpg
saved image 52.jpg
saved image 53.jpg
sa